In [1]:
from tqdm import tqdm_notebook;
import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA is available!  Training on GPU ...


In [2]:
DEVICE = torch.device("cuda")

In [3]:
!nvidia-smi
import torch
torch.cuda.is_available()

Fri Nov 25 23:08:10 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 522.06       Driver Version: 522.06       CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   46C    P0    19W /  N/A |   3169MiB /  4096MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [4]:
import torchvision
from torch.optim import lr_scheduler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from torchvision.transforms.transforms import Resize
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import pandas as pd
from matplotlib import colors, pyplot as plt
%matplotlib inline
from dataset import *
from utils import *
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [5]:
PATH_TO_LOAD = "./model101.pt"

In [6]:
n_classes = 11

In [7]:
class ResNet101(nn.Module):
    def __init__(self, pretrained=False):
        super(ResNet101, self).__init__()
        self.model = torchvision.models.resnet101(pretrained=pretrained)
        
        self.model.fc = nn.Linear(2048, n_classes)
        
                
                
    def forward(self, X):
        logits = self.model(X)
        if self.training:
            logits = self.linear(logits)
        return logits

In [8]:
resnet101 = ResNet101().to('cuda');

c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Aleksey Ryabykin\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [9]:
resnet101.model.load_state_dict(torch.load(PATH_TO_LOAD))

<All keys matched successfully>

In [11]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [12]:
def to_submit(test_files: list, test_dataset: CarsDataset, model, save=False) -> pd.DataFrame:
  test_dataset = CarsDataset(test_files, mode="test")
  test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)
  probs = predict(model, test_loader)


  preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
  test_filenames = [path.name for path in test_dataset.files]
  result = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
  if save:
    result.to_csv('submission.csv', index=False)
  return result

In [13]:
TEST_DIR = Path('public_test/')
PRIVATE_TEST_DIR = Path('private_dataset/')

test_files = sorted(list(TEST_DIR.rglob('*.jpg')))
test_dataset = CarsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64)

test_filenames = [path.name for path in test_dataset.files]

private_test_files = sorted(list(PRIVATE_TEST_DIR.rglob("*.jpg")))
private_test_dataset = CarsDataset(private_test_files, mode="test")
private_test_loader = DataLoader(private_test_dataset, shuffle=False, batch_size=64)

private_test_filenames = [path.name for path in private_test_dataset.files]


In [18]:
res = to_submit(private_test_files, private_test_dataset, resnet101)

In [19]:
res['numeric_id'] = res['Id'].apply(lambda row: int(row.split('.')[0]))
sorted_res = res.sort_values("numeric_id")
sorted_res.drop(columns=["Id", "numeric_id"]).to_csv("submission.csv", header=False, index=False)